# Features Engineering

Our data can be very high dimensional if we consider the number of languages, countries, actors etc. We have only 44k movies, but we have several hundreds on genres, languages and countries, which gives already around 10 millions possibilities. And this is without even looking at the hundreds of thousands of actors and characters. Thus we have to come up with features that can capture signal in the data, without having to do one-hot encoding for each feature.

## Packages

In [1]:
import pandas as pd
import pickle as pkl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Statistics
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf
# ML models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

## Helpers

In [2]:
CONTINENT_ID = {"North America and Australia": [1,0,0,0,0,0],
"Central and South America": [0,1,0,0,0,0],
"Western Europe": [0,0,1,0,0,0],
"Eastern Europe and Russia": [0,0,0,1,0,0],
"Africa and Middle-East": [0,0,0,0,1,0],
"Asia": [0,0,0,0,0,1]}

COUNTRY_CONTINENT_MAPPING = {
'afghanistan' : "Africa and Middle-East",
 'albania' : "Eastern Europe and Russia",
 'algeria' : "Africa and Middle-East",
 'argentina': "Central and South America",
 'armenia' : "Africa and Middle-East",
 'aruba': "Central and South America",
 'australia': "North America and Australia",
 'austria': "Western Europe",
 'azerbaijan' : "Africa and Middle-East",
 'bahamas': "Central and South America",
 'bahrain' : "Africa and Middle-East",
 'bangladesh': "Asia",
 'belgium': "Western Europe",
 'bhutan': "Asia",
 'bolivia': "Central and South America",
 'bosnia and herzegovina' : "Eastern Europe and Russia",
 'brazil': "Central and South America",
 'bulgaria' : "Eastern Europe and Russia",
 'burkina faso' : "Africa and Middle-East",
 'burma': "Asia",
 'cambodia': "Asia",
 'cameroon' : "Africa and Middle-East",
 'canada': "North America and Australia",
 'chile': "Central and South America",
 'china': "Asia",
 'colombia': "Central and South America",
 'congo' : "Africa and Middle-East",
 'costa rica': "Central and South America",
 'crime' : "Eastern Europe and Russia",
 'croatia' : "Eastern Europe and Russia",
 'cuba': "Central and South America",
 'cyprus' : "Eastern Europe and Russia",
 'czech republic' : "Eastern Europe and Russia",
 'czechoslovakia' : "Eastern Europe and Russia",
 'democratic republic of the congo' : "Africa and Middle-East",
 'denmark': "Western Europe",
 'egypt' : "Africa and Middle-East",
 'estonia' : "Eastern Europe and Russia",
 'ethiopia' : "Africa and Middle-East",
 'finland': "Western Europe",
 'france': "Western Europe",
 'georgia' : "Africa and Middle-East",
 'germany': "Western Europe",
 'greece' : "Eastern Europe and Russia",
 'haiti': "Central and South America",
 'hong kong': "Asia",
 'hungary' : "Eastern Europe and Russia",
 'iceland': "Western Europe",
 'india': "Asia",
 'indonesia': "Asia",
 'iran' : "Africa and Middle-East",
 'iraq' : "Africa and Middle-East",
 'iraqi kurdistan' : "Africa and Middle-East",
 'ireland': "Western Europe",
 'isle of man': "Western Europe",
 'israel' : "Africa and Middle-East",
 'italy': "Western Europe",
 'jamaica': "Central and South America",
 'japan': "Asia",
 'jordan' : "Africa and Middle-East",
 'kenya' : "Africa and Middle-East",
 'korea': "Asia",
 'kuwait' : "Africa and Middle-East",
 'lebanon' : "Africa and Middle-East",
 'libya' : "Africa and Middle-East",
 'lithuania' : "Eastern Europe and Russia",
 'luxembourg': "Western Europe",
 'macau': "Asia",
 'malaysia': "Asia",
 'mali' : "Africa and Middle-East",
 'malta': "Western Europe",
 'mexico': "Central and South America",
 'monaco': "Western Europe",
 'mongolia': "Asia",
 'montenegro' : "Eastern Europe and Russia",
 'morocco' : "Africa and Middle-East",
 'nepal': "Asia",
 'netherlands': "Western Europe",
 'new zealand': "North America and Australia",
 'nigeria' : "Africa and Middle-East",
 'norway': "Western Europe",
 'pakistan' : "Africa and Middle-East",
 'palestinian territories' : "Africa and Middle-East",
 'panama': "Central and South America",
 'peru': "Central and South America",
 'philippines': "Asia",
 'poland' : "Eastern Europe and Russia",
 'portugal': "Western Europe",
 'puerto rico': "Central and South America",
 'republic of macedonia' : "Eastern Europe and Russia",
 'romania' : "Eastern Europe and Russia",
 'russia' : "Eastern Europe and Russia",
 'senegal' : "Africa and Middle-East",
 'serbia' : "Eastern Europe and Russia",
 'serbia and montenegro' : "Eastern Europe and Russia",
 'singapore': "Asia",
 'slovakia' : "Eastern Europe and Russia",
 'slovenia' : "Eastern Europe and Russia",
 'south africa' : "Africa and Middle-East",
 'south korea': "Asia",
 'spain': "Western Europe",
 'sri lanka': "Asia",
 'sweden': "Western Europe",
 'switzerland': "Western Europe",
 'taiwan': "Asia",
 'thailand': "Asia",
 'tunisia' : "Africa and Middle-East",
 'turkey' : "Africa and Middle-East",
 'ukraine' : "Eastern Europe and Russia",
 'united arab emirates' : "Africa and Middle-East",
 'united kingdom': "Western Europe",
 'united states of america': "North America and Australia",
 'uruguay': "Central and South America",
 'uzbekistan' : "Africa and Middle-East",
 'venezuela': "Central and South America",
 'vietnam': "Asia",
 'yugoslavia' : "Eastern Europe and Russia",
 'zambia' : "Africa and Middle-East",
 'zimbabwe' : "Africa and Middle-East"
}

COUNTRY_ENCODING = { 
    "North America and Australia": [1,0,0,0,0,0],
    "Western Europe":              [0,1,0,0,0,0],
    "Asia":                        [0,0,1,0,0,0],
    "Africa and Middle-East":      [0,0,0,1,0,0],
    "Eastern Europe and Russia":   [0,0,0,0,1,0],
    "Central and South America":   [0,0,0,0,0,1]
}

CONTINENT_LIST = ["North America and Australia","Western Europe","Asia",
                  "Africa and Middle-East","Eastern Europe and Russia", 
                  "Central and South America"]

GENRE_MAPPING = {'absurdism': ["comedy"],
 'acid western': ["adventure","action"],
 'action': ["action"],
 'action comedy': ["action","comedy"],
 'action thrillers': ["thriller","action"],
 'action/adventure': ["action","adventure"],
 'addiction drama': ["drama"],
 'adult': ["adult"],
 'adventure': ["adventure"],
 'adventure comedy': ["adventure","comedy"],
 'airplanes and airports': ["other"],
 'albino bias': ["drama"],
 'alien film': ["action","adventure"],
 'alien invasion': ["action"],
 'americana': ["drama"],
 'animal picture': ["other"],
 'animals': ["other"],
 'animated cartoon': ["animation"],
 'animated musical': ["animation"],
 'animation': ["animation"],
 'anime': ["animation"],
 'anthology': ["genre"],
 'anthropology': ["other"],
 'anti-war': ["drama"],
 'anti-war film': ["drama"],
 'apocalyptic and post-apocalyptic fiction': ["action","fantasy"],
 'archaeology': ["other"],
 'archives and records': ["other"],
 'art film': ["genre"],
 'auto racing': ["other"],
 'avant-garde': ["genre"],
 'b-movie': ["comedy"],
 'b-western': ["action","comedy"],
 'backstage musical': ["other"],
 'baseball': ["other"],
 'beach film': ["other"],
 'beach party film': ["comedy"],
 'bengali cinema': ["other"],
 'biker film':["action"],
 'biographical film': ["other"],
 'biography': ["other"],
 'biopic [feature]': ["other"],
 'black comedy': ["comedy"],
 'black-and-white': ["other"],
 'blaxploitation':["drama"],
 'bloopers & candid camera': ["comedy"],
 'bollywood': ["other"],
 'boxing': ["other"],
 'breakdance': ["other"],
 'british empire film': ["other"],
 'british new wave': ["genre"],
 'bruceploitation':["action"],
 'buddy cop': ["action","comedy"],
 'buddy film': ["comedy"],
 'buddy picture': ["comedy"],
 'business': ["other"],
 'camp': ["other"],
 'caper story': ["thriller"],
 'cavalry film': ["action"],
 'chase movie': ["thriller"],
 'childhood drama': ["drama"],
 "children's": ["family"],
 "children's entertainment": ["family"],
 "children's fantasy": ["family","fantasy"],
 "children's issues": ["drama"],
 "children's/family": ["family"],
 'chinese movies': ["other"],
 'christian film': ["other"],
 'christmas movie': ["other"],
 'clay animation': ["animation"],
 'cold war': ["adventure","action"],
 'combat films': ["action"],
 'comdedy': ["comedy"],
 'comedy': ["comedy"],
 'comedy film': ["comedy"],
 'comedy horror': ["horror","comedy"],
 'comedy of errors': ["comedy"],
 'comedy of manners': ["comedy"],
 'comedy thriller': ["thriller","comedy"],
 'comedy western': ["action","comedy"],
 'comedy-drama': ["drama","comedy"],
 'coming of age': ["comedy"],
 'coming-of-age film': ["comedy"],
 'computer animation': ["animation"],
 'computers': ["animation"],
 'concert film': ["other"],
 'conspiracy fiction': ["thriller"],
 'costume adventure': ["adventure"],
 'costume drama': ["drama"],
 'costume horror': ["horror"],
 'courtroom comedy': ["comedy"],
 'courtroom drama': ["drama"],
 'creature film': ["adventure","fantasy"],
 'crime': ["thriller"],
 'crime comedy': ["thriller","comedy"],
 'crime drama': ["thriller","drama"],
 'crime fiction': ["thriller"],
 'crime thriller': ["thriller"],
 'cult': ["other"],
 'culture & society': ["other"],
 'cyberpunk': ["fantasy"],
 'czechoslovak new wave': ["genre"],
 'dance': ["other"],
 'demonic child': ["horror"],
 'detective': ["thriller"],
 'detective fiction': ["thriller"],
 'disaster': ["drama"],
 'docudrama': ["drama"],
 'documentary': ["other"],
 'dogme 95': ["genre"],
 'domestic comedy': ["comedy"],
 'doomsday film': ["fantasy"],
 'drama': ["drama"],
 'dystopia': ["drama","fantasy"],
 'ealing comedies': ["comedy"],
 'early black cinema': ["other"],
 'education': ["family"],
 'educational': ["family"],
 'ensemble film': ["genre"],
 'environmental science': ["other"],
 'epic': ["adventure"],
 'epic western': ["adventure","action"],
 'erotic drama': ["adult","drama"],
 'erotic thriller': ["thriller","adult"],
 'erotica': ["adult"],
 'escape film': ["thriller"],
 'essay film': ["genre"],
 'existentialism': ["genre"],
 'experimental film': ["genre"],
 'exploitation': ["drama"],
 'expressionism': ["genre"],
 'extreme sports': ["other"],
 'fairy tale': ["fantasy","adventure"],
 'family & personal relationships': ["drama"],
 'family drama': ["drama"],
 'family film': ["family"],
 'family-oriented adventure': ["family","adventure"],
 'fan film': ["other"],
 'fantasy': ["fantasy"],
 'fantasy adventure': ["fantasy","adventure"],
 'fantasy comedy': ["fantasy","comedy"],
 'fantasy drama': ["fantasy","drama"],
 'feature film': ["other"],
 'female buddy film': ["comedy"],
 'feminist film': ["drama"],
 'fictional film': ["other"],
 'filipino': ["other"],
 'filipino movies': ["other"],
 'film': ["other"],
 'film & television history': ["other"],
 'film adaptation': ["other"],
 'film noir': ["thriller"],
 'film à clef': ["drama"],
 'film-opera': ["other"],
 'filmed play': ["other"],
 'finance & investing': ["other"],
 'foreign legion':["action"],
 'future noir': ["fantasy","drama"],
 'gangster film': ["thriller","action"],
 'gay': ["drama"],
 'gay interest': ["drama"],
 'gay pornography': ["adult"],
 'gay themed': ["drama"],
 'gender issues': ["drama"],
 'giallo': ["thriller"],
 'glamorized spy film': ["thriller"],
 'goat gland': ["genre"],
 'gothic film': ["genre"],
 'graphic & applied arts': ["genre"],
 'gross out': ["comedy"],
 'gross-out film': ["comedy"],
 'gulf war':["action"],
 'hagiography': ["other"],
 'hardcore pornography': ["adult"],
 'haunted house film': ["horror"],
 'health & fitness': ["other"],
 'heaven-can-wait fantasies': ["fantasy"],
 'heavenly comedy': ["comedy"],
 'heist': ["action"],
 'hip hop movies': ["other"],
 'historical documentaries': ["other"],
 'historical drama': ["drama"],
 'historical epic': ["adventure"],
 'historical fiction': ["other"],
 'history': ["other"],
 'holiday film': ["comedy"],
 'horror': ["horror"],
 'horror comedy': ["horror","comedy"],
 'horse racing': ["other"],
 'humour': ["comedy"],
 'hybrid western': ["adventure","action"],
 'illnesses & disabilities': ["drama"],
 'indian western': ["adventure","action"],
 'indie': ["genre"],
 'inspirational drama': ["drama"],
 'instrumental music': ["other"],
 'interpersonal relationships': ["drama"],
 'inventions & innovations': ["other"],
 'japanese movies': ["other"],
 'journalism': ["other"],
 'jukebox musical': ["other"],
 'jungle film': ["adventure"],
 'juvenile delinquency film': ["drama"],
 'kafkaesque': ["genre"],
 'kitchen sink realism': ["genre"],
 'language & literature': ["genre"],
 'latino': ["other"],
 'law & crime': ["thriller"],
 'legal drama': ["drama"],
 'lgbt': ["drama"],
 'libraries and librarians': ["other"],
 'live action': ["other"],
 'malayalam cinema': ["other"],
 'marriage drama': ["drama"],
 'martial arts film': ["action"],
 'master criminal films': ["thriller"],
 'media satire': ["other"],
 'media studies': ["other"],
 'medical fiction': ["other"],
 'melodrama': ["drama"],
 'mockumentary': ["other"],
 'mondo film': ["genre"],
 'monster': ["horror","action"],
 'monster movie': ["horror","action"],
 'movie serial': ["other"],
 'movies about gladiators': ["other"],
 'mumblecore': ["genre"],
 'music': ["other"],
 'musical': ["other"],
 'musical comedy': ["comedy"],
 'musical drama': ["drama"],
 'mystery': ["thriller"],
 'mythological fantasy': ["fantasy"],
 'natural disaster': ["other"],
 'natural horror films': ["horror"],
 'nature': ["other"],
 'neo-noir': ["thriller"],
 'neorealism': ["genre"],
 'new hollywood': ["genre"],
 'new queer cinema': ["drama"],
 'news': ["other"],
 'ninja movie': ["action"],
 'northern': ["genre"],
 'operetta': ["other"],
 'outlaw': ["other"],
 'outlaw biker film': ["other"],
 'parkour in popular culture': ["action"],
 'parody': ["comedy"],
 'patriotic film': ["other"],
 'period horror': ["horror"],
 'period piece': ["drama"],
 'pinku eiga': ["adult"],
 'plague': ["drama"],
 'point of view shot': ["other"],
 'political cinema': ["other"],
 'political documetary': ["other"],
 'political drama': ["drama"],
 'political satire': ["drama"],
 'political thriller': ["thriller"],
 'pornographic movie': ["adult"],
 'pornography': ["adult"],
 'pre-code': ["other"],
 'prison': ["action"],
 'prison escape': ["action"],
 'prison film': ["action"],
 'private military company': ["action"],
 'propaganda film': ["other"],
 'psycho-biddy': ["horror","thriller"],
 'psychological horror': ["horror"],
 'psychological thriller': ["thriller"],
 'punk rock': ["genre"],
 'race movie': ["action"],
 'reboot': ["other"],
 'religious film': ["other"],
 'remake': ["other"],
 'revenge': ["action"],
 'revisionist fairy tale': ["adventure","fantasy"],
 'revisionist western': ["adventure","action"],
 'road movie': ["other"],
 'road-horror': ["horror"],
 'roadshow theatrical release': ["other"],
 'roadshow/carny': ["other"],
 'rockumentary': ["other"],
 'romance film': ["other"],
 'romantic comedy': ["comedy"],
 'romantic drama': ["drama"],
 'romantic fantasy': ["fantasy"],
 'romantic thriller': ["thriller"],
 'samurai cinema': ["adventure","action"],
 'satire': ["comedy"],
 'school story': ["family"],
 'sci-fi adventure': ["fantasy","adventure"],
 'sci-fi horror': ["fantasy","horror"],
 'sci-fi thriller': ["fantasy","thriller"],
 'science fiction': ["fantasy"],
 'science fiction western': ["adventure","fantasy","action"],
 'screwball comedy': ["comedy"],
 'sex comedy': ["comedy"],
 'sexploitation': ["drama"],
 'short film': ["other"],
 'silent film': ["other"],
 'singing cowboy': ["action","adventure"],
 'slapstick': ["comedy"],
 'slasher': ["horror","action"],
 'slice of life story': ["drama"],
 'social issues': ["drama"],
 'social problem film': ["drama"],
 'softcore porn': ["adult"],
 'space opera': ["fantasy","adventure"],
 'space western': ["adventure","action"],
 'spaghetti western': ["adventure","action"],
 'splatter film': ["horror"],
 'sports': ["other"],
 'spy': ["thriller"],
 'stand-up comedy': ["comedy"],
 'star vehicle': ["other"],
 'steampunk': ["fantasy"],
 'stoner film': ["genre"],
 'stop motion': ["animation"],
 'superhero': ["action"],
 'superhero movie': ["action"],
 'supermarionation': ["animation"],
 'supernatural': ["fantasy"],
 'surrealism': ["genre"],
 'suspense': ["thriller"],
 'swashbuckler films': ["action","adventure"],
 'sword and sandal': ["adventure"],
 'sword and sorcery': ["fantasy","adventure"],
 'sword and sorcery films': ["fantasy","adventure"],
 'tamil cinema': ["other"],
 'teen': ["family"],
 'television movie': ["other"],
 'the netherlands in world war ii': ["action"],
 'therimin music': ["other"],
 'thriller': ["thriller"],
 'time travel': ["adventure"],
 'tokusatsu': ["other"],
 'tollywood': ["other"],
 'tragedy': ["drama"],
 'tragicomedy': ["comedy"],
 'travel': ["adventure"],
 'vampire movies': ["horror","action"],
 'war effort': ["action"],
 'war film': ["action"],
 'werewolf fiction': ["horror"],
 'western': ["adventure","action"],
 'whodunit': ["thriller"],
 'women in prison films': ["drama"],
 'workplace comedy': ["comedy"],
 'world cinema': ["other"],
 'world history': ["other"],
 'wuxia': ["adventure"],
 'z movie': ["horror"],
 'zombie film': ["horror"]}

GENRE_ENCODING = {
    "action":    [1,0,0,0,0,0,0,0,0,0,0],
    "adventure": [0,1,0,0,0,0,0,0,0,0,0],
    "comedy":    [0,0,1,0,0,0,0,0,0,0,0],
    "drama":     [0,0,0,1,0,0,0,0,0,0,0],
    "thriller":  [0,0,0,0,1,0,0,0,0,0,0],
    "horror":    [0,0,0,0,0,1,0,0,0,0,0],
    "animation": [0,0,0,0,0,0,1,0,0,0,0],
    "family":    [0,0,0,0,0,0,0,1,0,0,0],
    "adult":     [0,0,0,0,0,0,0,0,1,0,0],
    "fantasy":   [0,0,0,0,0,0,0,0,0,1,0],
    "genre":     [0,0,0,0,0,0,0,0,0,0,1],
    "other":     [0,0,0,0,0,0,0,0,0,0,0]
}

GENRE_LIST = ["action","adventure","comedy","drama","thriller","horror",
                   "animation","children","adult","fantasy","genre"]

DEFAULT_PARAMETERS = {"drop": ["name","revenue","has_common_character_name","character_number"],
              "nan_filtering":["all"],
              "decades":[],
              "log":[],
              "standardize":["title_length"]}

SUCCESS_THRESHOLD = 7.5

## Load Data

In [3]:
country_df = pd.read_pickle("../../data/post_processing//country_df.pkl")
comes_from_df = pd.read_pickle("../../data/post_processing/comes_from_df.pkl")
genre_df = pd.read_pickle("../../data/post_processing/genre_df.pkl")
is_of_type_df = pd.read_pickle("../../data/post_processing/is_of_type_df.pkl")
language_df = pd.read_pickle("../../data/post_processing/language_df.pkl")
spoken_languages_df = pd.read_pickle("../../data/post_processing/spoken_languages_df.pkl")
character_df = pd.read_pickle("../../data/post_processing/character_df.pkl")
actor_df = pd.read_pickle("../../data/post_processing/actor_df.pkl")
movie_df = pd.read_pickle("../../data/post_processing/movie_df.pkl")
belongs_to_df = pd.read_pickle("../../data/post_processing/belongs_to_df.pkl")
play_df = pd.read_pickle("../../data/post_processing/play_df.pkl")
appears_in_df = pd.read_pickle("../../data/post_processing/appears_in_df.pkl")
wikipedia_imdb_mapping_table = pd.read_pickle("../../data/generated/wikipedia_imdb_mapping_df.pkl")
is_directed_by_df = pd.read_pickle("../../data/post_processing/is_directed_by_df.pkl")
director_df = pd.read_pickle("../../data/post_processing/director_df.pkl")

## Create Regression DataFrame

In [4]:
movie_regression_df = movie_df.copy()
movie_regression_df.drop(["freebase_id","plot"],axis=1,inplace=True)
movie_regression_df["num_votes"] = movie_regression_df["num_votes"].astype(np.int32)

## Countries

For the countries we can already try to group per continent. It will give an idea of the different local influence without going in too much details. We suggest the following partition based on our previous analyses:

- North America and Australia
- Central and South America
- Western Europe
- Eastern Europe and Russia
- Africa and Middle-East
- Asia

In [5]:
country_movie_df = comes_from_df.copy()
country_movie_df["country_encoding"] = country_movie_df["country_name"].apply(
    lambda c: COUNTRY_CONTINENT_MAPPING[c])
country_movie_df["country_encoding"] = country_movie_df["country_encoding"].apply(
    lambda c: np.array(COUNTRY_ENCODING[c]))
for continent in CONTINENT_LIST:
    country_movie_df[continent] = country_movie_df["country_encoding"].apply(
        lambda l: l[CONTINENT_LIST.index(continent)])
    movie_regression_df[continent] = country_movie_df.groupby("movie_id")[continent].max()

## Actors

For the actors, we thought about the following features:
- #number of actors in the top 5% (or 1% ?)
- #number of actors
- mean actor age
- gender balance

In [6]:
percentile = 99
actor_movie_count = appears_in_df.groupby("actor_id")["movie_id"].count().values
threshold = np.percentile(actor_movie_count,percentile)
is_actor_above_threshold = appears_in_df.groupby("actor_id")["movie_id"].count().sort_index() >= threshold
top_k_actors = actor_df[is_actor_above_threshold.values]
print(f"There are {len(top_k_actors)} actors in the top {percentile}%.")

There are 1014 actors in the top 99%.


In [7]:
actor_movie_df = appears_in_df.merge(actor_df["gender"],how="left",on="actor_id")
actor_movie_df["gender"] = actor_movie_df["gender"].apply(
    lambda g: -1 if g == "M" else 1 if g == "F" else g)
actor_movie_df["is_top_k"] = actor_movie_df["actor_id"].isin(set(top_k_actors.index))

In [8]:
movie_regression_df["actor_number"] = actor_movie_df.groupby("movie_id")["actor_id"].count()
movie_regression_df["mean_actor_age"] = actor_movie_df.groupby("movie_id")["actor_age"].mean()
movie_regression_df["gender_ratio"] = actor_movie_df.groupby("movie_id")["gender"].mean()
movie_regression_df["has_famous_actor"] = actor_movie_df.groupby("movie_id")["is_top_k"].max()
movie_regression_df["has_famous_actor"] = movie_regression_df["has_famous_actor"].replace({True: 1, False: 0})

## Genre

For the genre we suggest the following features:
- #genres
- One-hot-encoding of genre cluster:
    - action
    - adventure
    - comedy
    - drame
    - thriller
    - horror
    - animation
    - children
    - adult
    - fantasy
    - genre
    - other

In [9]:
genre_movie_df = is_of_type_df.copy()
genre_movie_df["genre_encoding"] = genre_movie_df["genre_name"].apply(lambda g: GENRE_MAPPING[g])
genre_movie_df["genre_encoding"] = genre_movie_df["genre_encoding"].apply(
    lambda l: np.sum([np.array(GENRE_ENCODING[g]) for g in l],axis=0))
for genre in GENRE_LIST:
    genre_movie_df[genre] = genre_movie_df["genre_encoding"].apply(
        lambda l: l[GENRE_LIST.index(genre)])
    movie_regression_df[genre] = genre_movie_df.groupby("movie_id")[genre].max()
movie_regression_df["genre_number"] = genre_movie_df.groupby("movie_id")["genre_name"].count()

## Languages

For the languages we suggest the following features:
- #languages
- top languages
- total population covered?

In [10]:
k_language = 5
language_movie_df = spoken_languages_df.copy()
top_k_languages = set(language_movie_df.groupby("language_name")["movie_id"].count().sort_values(
    ascending=False).head(k_language).index)
language_movie_df["top_language"] = language_movie_df["language_name"].isin(top_k_languages)
movie_regression_df["has_common_language"] = language_movie_df.groupby("movie_id")["top_language"].max()
movie_regression_df["has_common_language"] = movie_regression_df["has_common_language"].replace({True: 1, False: 0})
movie_regression_df["language_number"] = language_movie_df.groupby("movie_id")["language_name"].count()

## Characters
- #characters
- usage of common names

In [11]:
k_characters = 20
top_k_characters_names = set(character_df["character_name"].value_counts().head(k_characters).index)
top_k_characters_ids = set(character_df[character_df["character_name"].isin(top_k_characters_names)].index)
character_movie_df = belongs_to_df.copy()
character_movie_df["common_character_name"] = character_movie_df["character_id"].isin(top_k_characters_ids)
movie_regression_df["has_common_character_name"] = character_movie_df.groupby(
    "movie_id")["common_character_name"].max()
movie_regression_df["character_number"] = character_movie_df.groupby(
    "movie_id")["character_id"].count()
movie_regression_df["has_common_character_name"] = movie_regression_df[
    "has_common_character_name"].replace({True: 1, False: 0})

## Decade

In [12]:
movie_regression_df["decade"] = movie_regression_df["release_date"].apply(lambda d: d.year - d.year%10)

## Movie Title

In [13]:
movie_regression_df["title_length"] = movie_regression_df["name"].apply(lambda n: len(n.split()))

## Director

- Has the director produced a successful movie in the past decades
- Has the director produced a successful movie in the past (coarser feature)
- Number of movies produced by directors in the past

- Number of directors on the movie

In [14]:
def director_metrics_up_to_date(movie_dataframe: pd.DataFrame, director_movies: set, date) -> tuple:
    """
    Compute the number of movies and the best ratings up to the date from the set of movies provided.
    
    This function allows us to have the history of performance of the set of directors that worked 
    on a given movie. You give the set of movies of the directors and the date of production of the
    current movie and it will return the performance up to the given date.
    
    :param movie_dataframe: Pandas DataFrame with movie information
    :param director_movies: Set of movies directed by the directors of the current movie.
    :param date: Date of release of the current movie, thus the metrics are up to this date.
    
    :return: The number of movies and the best rating up to the given date.
    """
    director_movie_df = movie_dataframe[movie_dataframe.index.isin(director_movies)]
    movie_date_df = director_movie_df[director_movie_df["release_date"] < date]
    movie_count = len(movie_date_df)
    if movie_count == 0:
        return 0, None
    best_rating = movie_date_df["average_rating"].max()
    return movie_count, best_rating

In [15]:
director_movies_mapping = is_directed_by_df.groupby("director_id")["movie_id"].apply(set).to_dict()

In [16]:
# Compute the set of movies directed by the directors for each movie
director_movies_df = is_directed_by_df.copy()
director_movies_df["director_movies"] = director_movies_df["director_id"].apply(
    lambda d: director_movies_mapping[d])
director_movies_df = director_movies_df.groupby("movie_id")["director_movies"].apply(list)
director_movies_df = director_movies_df.apply(
    lambda movie_list: set([m for movie_set in movie_list for m in list(movie_set)]))
director_movies_df = pd.DataFrame(director_movies_df)
# Merge DataFrames to recover temporal data and rating
new_is_directed_by_df = director_movies_df.merge(
    movie_regression_df[["average_rating","release_date"]],how="left",on="movie_id")
new_is_directed_by_df["combined_features"] = list(zip(new_is_directed_by_df["director_movies"],new_is_directed_by_df["release_date"]))
# Compute performance of the movie directing team up to the release date of each movie.
new_is_directed_by_df["director_metrics"] = new_is_directed_by_df["combined_features"].apply(
    lambda t: director_metrics_up_to_date(new_is_directed_by_df,t[0],t[1]))

In [17]:
movie_regression_df["combinned_movie_num"] = new_is_directed_by_df["director_metrics"].apply(lambda t: t[0])
movie_regression_df["combinned_best_rating"] = new_is_directed_by_df["director_metrics"].apply(lambda t: t[1])
movie_regression_df["num_directors"] = is_directed_by_df.groupby("movie_id").count()

In [18]:
movie_regression_df["combinned_movie_success"] = movie_regression_df["combinned_best_rating"] > SUCCESS_THRESHOLD
movie_regression_df["combinned_movie_success"] = movie_regression_df[
    "combinned_movie_success"].replace({True: 1, False: 0})

## Plot

- Preprocess plots: tokenization + casefolding + stopwords removal + lemmatization
- BOW creation
- BOW matrix creation
- odds ratio computation

### Plot Preprocessing

In [66]:
# imports 
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jerem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# Tokenize
movies_with_plot_df = pd.DataFrame(movie_df[~movie_df["plot"].isna()]["plot"])
movies_with_plot_df["tokenized"] = movies_with_plot_df["plot"].apply(tokenizer.tokenize)
movies_with_plot_df.head(3)

,plot,tokenized
movie_id,,
975900,"Set in the second half of the 22nd century, th...","[Set, in, the, second, half, of, the, 22nd, ce..."
261236,"Eva, an upper class housewife, becomes frustra...","[Eva, an, upper, class, housewife, becomes, fr..."
171005,{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...,"[Plot, dateAct, 1Act, 2Act, 3Act, 4Act, 5, Fin..."


In [68]:
# Stopwords and Casefolding
movies_with_plot_df["no_stopwords"] = movies_with_plot_df["tokenized"].apply(
    lambda l: [s.casefold() for s in l if s.casefold() not in stop_words])
movies_with_plot_df.head(3)

,plot,tokenized,no_stopwords
movie_id,,,
975900,"Set in the second half of the 22nd century, th...","[Set, in, the, second, half, of, the, 22nd, ce...","[set, second, half, 22nd, century, film, depic..."
261236,"Eva, an upper class housewife, becomes frustra...","[Eva, an, upper, class, housewife, becomes, fr...","[eva, upper, class, housewife, becomes, frustr..."
171005,{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...,"[Plot, dateAct, 1Act, 2Act, 3Act, 4Act, 5, Fin...","[plot, dateact, 1act, 2act, 3act, 4act, 5, fin..."


In [69]:
# Lemmatize
movies_with_plot_df["lemmatized"] = movies_with_plot_df["no_stopwords"].apply(
    lambda l: [lemmatizer.lemmatize(s) for s in l])
movies_with_plot_df.head(3)

,plot,tokenized,no_stopwords,lemmatized
movie_id,,,,
975900,"Set in the second half of the 22nd century, th...","[Set, in, the, second, half, of, the, 22nd, ce...","[set, second, half, 22nd, century, film, depic...","[set, second, half, 22nd, century, film, depic..."
261236,"Eva, an upper class housewife, becomes frustra...","[Eva, an, upper, class, housewife, becomes, fr...","[eva, upper, class, housewife, becomes, frustr...","[eva, upper, class, housewife, becomes, frustr..."
171005,{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...,"[Plot, dateAct, 1Act, 2Act, 3Act, 4Act, 5, Fin...","[plot, dateact, 1act, 2act, 3act, 4act, 5, fin...","[plot, dateact, 1act, 2act, 3act, 4act, 5, fin..."


### BOW creation

In [82]:
BOW = set()
for row_id, row_data in movies_with_plot_df["lemmatized"].apply(lambda l: set(l)).iteritems():
    BOW = BOW.union(row_data)

### BOW Matrix

In [132]:
def BOW_mapping(words_list: list, BOW_dict: dict) -> dict:
    """
    Map the BOW for each plot to its encoding in row id and number of occurences.
    
    :param words_list: List of the words contained in the plot (BOW).
    :param BOW_dict: Dictionnary containing the mapping between words and ids in the BOW matrix.
    
    :return: The dictionnary with the ids of each words and their number of occurences.
    
    """
    movie_mapping = dict()
    for w in words_list:
        word_id = BOW_dict[w]
        if word_id not in movie_mapping:
            movie_mapping[word_id] = 1
        else:
            movie_mapping[word_id] += 1 
    return movie_mapping

In [133]:
BOW_dict = dict(zip(BOW,range(len(BOW))))

In [134]:
movies_with_plot_df["encoding"] = movies_with_plot_df["lemmatized"].apply(
    lambda l: BOW_mapping(l,BOW_dict))
movies_with_plot_df.head(3)

,plot,tokenized,no_stopwords,lemmatized,encoding
movie_id,,,,,
975900,"Set in the second half of the 22nd century, th...","[Set, in, the, second, half, of, the, 22nd, ce...","[set, second, half, 22nd, century, film, depic...","[set, second, half, 22nd, century, film, depic...","{58165: 2, 75398: 2, 4156: 1, 66141: 1, 15297:..."
261236,"Eva, an upper class housewife, becomes frustra...","[Eva, an, upper, class, housewife, becomes, fr...","[eva, upper, class, housewife, becomes, frustr...","[eva, upper, class, housewife, becomes, frustr...","{95024: 5, 367: 1, 67450: 1, 10939: 1, 80455: ..."
171005,{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...,"[Plot, dateAct, 1Act, 2Act, 3Act, 4Act, 5, Fin...","[plot, dateact, 1act, 2act, 3act, 4act, 5, fin...","[plot, dateact, 1act, 2act, 3act, 4act, 5, fin...","{86899: 1, 61096: 1, 14211: 1, 74318: 1, 88368..."


In [138]:
column_ids = list(range(len(movies_with_plot_df)))
row_ids = movies_with_plot_df["encoding"].apply(lambda d: list(d.keys()))
data = movies_with_plot_df["encoding"].apply(lambda d: list(d.values()))

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
pd.Series([[1,2,3],[1,2],[1,2,3,4]])

In [140]:
csr_matrix(data, (column_ids,row_ids))

TypeError: no supported conversion for types: (dtype('O'),)

## Regression

In [19]:
def standardize_column(dataframe: pd.DataFrame, col_name: str):
    """
    Standardize the given column in the provided dataframe.
    
    :param dataframe: Pandas DataFrame containing the data.
    :param col_name: Name of the column to standardize.
    
    """
    dataframe[col_name] = (dataframe[col_name]-
        dataframe[col_name].mean())/dataframe[col_name].std()
    
def process_dataframe(dataframe: pd.DataFrame,
                      parameters={"drop": [], "nan_filtering":["all"],"decades":[],
                                  "log":[], "standardize":[]}) -> pd.DataFrame:
    """
    Pre-process the dataframe for regression given the instructions in parameters.
    
    :param dataframe: Pandas DataFrame with the movie data for regression
    :param parameters: Dictionnary with the different parameters for pre-processing:
                            - drop: List of columns to drop.
                            - nan_filtering: List of columns where nan rows should be excluded.
                            - decades: List of decades to keep. If empty, keep all decades.
                            - log: List of columns to log transform.
                            - standardize: List of columns to standardize.
                    
    
    :return: Processed Pandas DataFrame ready for regression.
    
    """
    regression_df = dataframe.copy()
    # Filter out columns
    regression_df = regression_df.drop(parameters["drop"],axis=1)
    # Filter out decades
    if len(parameters["decades"]) != 0:
        regression_df = regression_df[
            regression_df["decade"].isin(parameters["decades"])]
    # Filter out NaN
    if len(parameters["nan_filtering"]) != 0:
        if parameters["nan_filtering"][0] == "all":
            regression_df = regression_df.dropna(how="any")
        else:
            regression_df = regression_df.dropna(subset=parameters["nan_filtering"])
    # Transform
    for col in parameters["log"]:
        regression_df["log_"+col] = regression_df[col].apply(np.log)
    # Standardize
    for col in parameters["standardize"]:
        standardize_column(regression_df,col)
    return regression_df

In [20]:
def select_next_feature(regression_df: pd.DataFrame, target: pd.Series,
                        current_features=[], ignored_features=[], 
                        alpha=0.05, show=False) -> str:
    """
    Report the best feature to integrate to the current OLS model based on r-squared.
    
    :param regression_df: Pandas DataFrame containing the data for regression.
    :param target: Pandas Series representing the target values.
    :param current_features: List of features integrated in the actual model.
    :param ignored_features: List of features that should not be integrated in the regression.
    :param alpha: Significance level, default 0.05.
    :param show: Display the different features scores.
    
    :result: Name of the best feature or None if no new significant features.
    
    """
    result_dict = {'predictor': [], 'r-squared':[], "aic":[], "p_value":[]}
    for col in regression_df.columns:
        if col not in (current_features+ignored_features):
            X = regression_df[current_features + [col]]
            model = sm.OLS(target, sm.add_constant(X)).fit()
            #Add the column name to our dictionary
            result_dict['predictor'].append(col)
            #Calculate the r-squared value between the target and predicted target
            r2 = model.rsquared
            #Add the model metrics to our dictionary
            result_dict['r-squared'].append(r2)
            result_dict['aic'].append(model.aic)
            result_dict['p_value'].append(model.pvalues.loc[col])
    #Once it's iterated through every column, turn our dict into a sorted DataFrame
    candidates_features = pd.DataFrame(result_dict).sort_values(by=['r-squared'],
                                                          ascending = False)
    if show:
        display(candidates_features.head())
        
    candidates_features = candidates_features[candidates_features["p_value"] < alpha]
    if len(candidates_features) == 0:
        return "None"
    else:
        return candidates_features["predictor"].iloc[0]

In [21]:
def forward_selection(regression_df: pd.DataFrame, target: pd.Series,
                        ignored_features=[], alpha=0.05, show=False) -> list:
    """
    Iterative forward feature selection based on r-squared without interaction terms.
    
    :param regression_df: Pandas DataFrame containing the data for regression.
    :param target: Pandas Series representing the target values.
    :param current_features: List of features integrated in the actual model.
    :param ignored_features: List of features that should not be integrated in the regression.
    :param alpha: Significance level, default 0.05.
    :param show: Display the different features scores.
    
    :result: List of the best features to model the target.
    
    """
    last_feature = ""
    current_features = []
    while ((len(ignored_features) + len(current_features)) < len(regression_df)
           and last_feature != "None"):
        last_feature = select_next_feature(regression_df, target,
                        current_features=current_features,
                        ignored_features=ignored_features, 
                        alpha=alpha, show=show)
        if last_feature != "None":
            current_features.append(last_feature)
    return current_features

In [22]:
def create_VIF_dataframe(regression_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute the VIF for each features in the given dataframe.
    
    :param regression_df: Pandas DataFrame containing the data for regression.
    
    :return: Pandas DataFrame with the VIF for each features.
    
    """
    vif_features = regression_df.columns
    vif_values = [variance_inflation_factor(regression_df.values, i) 
                for i in range(len(regression_df.columns))]
    vif_df = pd.DataFrame(np.array([vif_features,vif_values]).T,columns=["predictor","VIF"])
    return vif_df

def filter_multicolinearity(regression_df: pd.DataFrame, threshold=5):
    """
    Remove the features that shows high multicollinearity based on VIF.
    
    :param regression_df: Pandas DataFrame containing the data for regression.
    :param threshold: Threshold above which a VIF is considered to high to be kept. 
    
    :return: Pandas DataFrame with data for regression without high multicollinearity.
    
    """
    new_regression_df = regression_df.copy()
    vif_df = create_VIF_dataframe(new_regression_df)
    high_VIF = (vif_df["VIF"] > threshold).sum()
    while high_VIF:
        highest_VIF_predictor = vif_df.iloc[
            vif_df["VIF"].astype(np.float32).argmax()]["predictor"]
        new_regression_df = new_regression_df.drop(highest_VIF_predictor,axis=1)
        vif_df = create_VIF_dataframe(new_regression_df)
        high_VIF = (vif_df["VIF"] > threshold).sum()
    return new_regression_df, vif_df

In [23]:
def format_regression_df(dataframe: pd.DataFrame, decades: list,
                         parameters=DEFAULT_PARAMETERS, target_threshold=8) -> tuple:
    """
    Process the dataframe for regression and creates targets and weights vectors.
    
    :param dataframe: Pandas DataFrame containing the data for regression.
    :param decades: List of decades to integrate for regression. 
                    If empty then all decades will be considered.
    :param parameters: Parameter dictionnary to process the dataframe.
    :param target_threshold: Threshold from which we consider a movie as successful.
      
    :return: Tuple with the regression DataFrame, the raw and binary targets and the number of votes.
    """
    parameters["decades"] = decades
    processed_df = process_dataframe(dataframe,parameters)
    # Extract target and associated features.
    target, num_votes= processed_df["average_rating"], processed_df["num_votes"]
    binary_target = target.copy()
    binary_target[binary_target<target_threshold] = 0
    binary_target[binary_target>=target_threshold] = 1
    # Remove Unecessary Columns.
    processed_df = processed_df.drop(["release_date","num_votes",
                                "runtime","has_common_language","language_number",
                                "decade","average_rating","combinned_best_rating"],axis=1)
    processed_df, vif_df = filter_multicolinearity(processed_df)
    return processed_df, target, binary_target, num_votes

In [24]:
processed_df, target, binary_target, num_votes = format_regression_df(movie_regression_df,[])

In [25]:
features = forward_selection(processed_df, target, ignored_features=[], alpha=0.05, show=False)

In [26]:
model = sm.OLS(target, sm.add_constant(processed_df[features])).fit()

In [27]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         average_rating   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     201.9
Date:                Fri, 16 Dec 2022   Prob (F-statistic):               0.00
Time:                        11:05:58   Log-Likelihood:                -24472.
No. Observations:               18482   AIC:                         4.898e+04
Df Residuals:                   18463   BIC:                         4.913e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         5.8532      0.020    293.682      0.000       5.814       5.892
combinned_movie_success       0.4828      0.015     32.604      0.000       0.454       0.512
drama                         0.3198      0.015     20.836      0.000       0.290       0.350
horror                       -0.3333      0.027    -12.172      0.000      -0.387      -0.280
Western Europe                0.1627      0.016     10.298      0.000       0.132       0.194
animation                     0.5907      0.054     10.893      0.000       0.484       0.697
fantasy                      -0.2257      0.024     -9.358      0.000      -0.273      -0.178
actor_number                  0.0148      0.001     11.703      0.000       0.012       0.017
Eastern Europe and Russia     0.3215      0.045      7.199      0.000       0.234       0.409
Asia                          0.1356      0.019      7.093      0.000       0.098       0.173
adult                        -0.4303      0.061     -7.082      0.000      -0.549      -0.311
action                       -0.0972      0.020     -4.918      0.000      -0.136      -0.058
children                     -0.1515      0.028     -5.468      0.000      -0.206      -0.097
comedy                       -0.0986      0.016     -6.300      0.000      -0.129      -0.068
genre                         0.1294      0.022      5.848      0.000       0.086       0.173
gender_ratio                 -0.0854      0.017     -4.937      0.000      -0.119      -0.052
adventure                    -0.0654      0.021     -3.099      0.002      -0.107      -0.024
thriller                     -0.0425      0.016     -2.705      0.007      -0.073      -0.012
title_length                  0.0169      0.007      2.477      0.013       0.004       0.030
==============================================================================
Omnibus:                     2191.820   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3803.930
Skew:                          -0.807   Prob(JB):                         0.00
Kurtosis:                       4.528   Cond. No.                         98.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Decade pipeline

In [ ]:
decade_results = dict()
for decade in movie_regression_df["decade"].value_counts().head(11).index:
    processed_df, target, binary_target, num_votes = format_regression_df(movie_regression_df,[decade])
    features = forward_selection(processed_df, target, ignored_features=[], alpha=0.05, show=False)
    model = sm.OLS(target, sm.add_constant(processed_df)).fit()
    decade_results[decade] = model.rsquared
decade_results_df = pd.DataFrame(decade_results.values(),index=decade_results.keys(),
                                columns=["r-squared"]).sort_index()

In [ ]:
decade_results_df

### Logistic Regression

In [ ]:
"""
model = sm.GLM(binary_target, sm.add_constant(process_df[current_features]),
               family=sm.families.Binomial(link=sm.families.links.logit())).fit_regularized(
                alpha=0.001,L1_wt=0)
""";

In [ ]:
model = sm.Logit(binary_target, sm.add_constant(process_df)).fit()
model.summary()

In [ ]:
tp = binary_target[model.predict(sm.add_constant(process_df)) > 0.5].sum()
print(f"The Logistic Regression models with all features gives us {tp:.0f} true positives.")

---

### Weighted Regression

In [ ]:
log_num_votes = np.log(num_votes)
weights = 1+(log_num_votes-log_num_votes.min())/(log_num_votes.max()-log_num_votes.min())

In [ ]:
regr = LinearRegression()
regr.fit(process_df, target,
         weights);

---

### Random Forest

In [ ]:
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [ ]:
tmp_df = process_df.copy()
tmp_df["target"] = binary_target.copy()
train, test = train_test_split(tmp_df, test_size=0.2)

In [ ]:
clf = RandomForestClassifier()
_ = clf.fit(train.drop("target",axis=1), train["target"])

In [ ]:
preds = clf.predict(test.drop("target",axis=1))
score = metrics.f1_score(test["target"], preds)

In [ ]:
score

## Advanced Tasks

- Compute centrality of directors and actors instead of #movies
- Subset on plot and extract NLP features --> odds ratio of terms / LDA and topic association (decorelation with genre)
- Matrix factorization for value imputation

In [60]:
import networkx as nx

In [75]:
B = nx.Graph()
B.add_nodes_from(set(appears_in_df["actor_id"]), bipartite=0)
B.add_nodes_from(set(appears_in_df["movie_id"]), bipartite=1)
actor_movie_edges = list(zip(appears_in_df["actor_id"],appears_in_df["movie_id"]))
B.add_edges_from(actor_movie_edges)

In [80]:
actor_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}
movie_nodes = set(B) - top_nodes

In [83]:
actor_graph = nx.projected_graph(B,actor_nodes)

In [84]:
actor_centrality = nx.harmonic_centrality(actor_graph,actor_nodes)

KeyboardInterrupt: 

In [35]:
movie_df.merge(movie_regression_df["decade"],on="movie_id",how="left").groupby("decade")["plot"].count()

decade
1890       1
1900       5
1910     124
1920     432
1930    1340
1940    1503
1950    1851
1960    1858
1970    2122
1980    2973
1990    3995
2000    7791
2010    2292
2020       0
Name: plot, dtype: int64